Define Daubechy wavelet transformation function based on percentage and wavelet

In [ ]:
import numpy as np
import cv2
import pywt
import os
from numpy.fft import fft2, fftshift, ifft2, ifftshift

def zero_coefficients(coeffs, percentage):
    cA, (cH, cV, cD) = coeffs
    all_coeffs = np.hstack([c.flatten() for c in [cA, cH, cV, cD]])
    threshold = np.percentile(np.abs(all_coeffs), percentage)

    cA = np.where(np.abs(cA) < threshold, 0, cA)
    cH = np.where(np.abs(cH) < threshold, 0, cH)
    cV = np.where(np.abs(cV) < threshold, 0, cV)
    cD = np.where(np.abs(cD) < threshold, 0, cD)

    return cA, (cH, cV, cD)

def apply_dwt_and_zero_coefficients_multidim(image, wavelet_name, percentage):
    channels = cv2.split(image)
    result_channels = []

    for channel in channels:
        coeffs2 = pywt.dwt2(channel, wavelet_name)
        coeffs2_zeroed = zero_coefficients(coeffs2, percentage)
        processed_channel = pywt.idwt2(coeffs2_zeroed, wavelet_name)
        result_channels.append(processed_channel)

    processed_image = cv2.merge(result_channels)
    return processed_image

Define Fourier function transformation function based on percentage

In [ ]:
def fft_filter_on_color_image(image, percentage):
    channels = cv2.split(image)
    result_channels = []

    for channel in channels:
        # Apply FFT
        f = fft2(channel)
        fshift = fftshift(f)

        rows, cols = channel.shape
        crow, ccol = rows // 2, cols // 2

        mask_size = int(min(rows, cols) * percentage / 100 / 2)
        mask = np.zeros((rows, cols), np.uint8)
        mask[crow-mask_size:crow+mask_size, ccol-mask_size:ccol+mask_size] = 1

        fshift_masked = fshift * mask
        f_ishift = ifft2(ifftshift(fshift_masked))
        img_back = np.abs(f_ishift)

        result_channels.append(img_back)

    processed_image = cv2.merge(result_channels)
    return processed_image

In [ ]:
HOME = os.getcwd()
print("HOME:", HOME)

Apply DWT to the segmented image

In [ ]:
percentage = ...# choose from 1,5,20
wavelet = ...# choose from db1, db2, db3

dataset_path = os.path.join(HOME, 'Original_segmentation', 'train')
output_path = os.path.join(HOME, f'DWT_{wavelet}_{perecntage}%_segmentation', 'train')

os.makedirs(output_path, exist_ok=True)



for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)
    target_folder_path = os.path.join(output_path, folder_name)

    os.makedirs(target_folder_path, exist_ok=True)

    for image_filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_filename)
        image = cv2.imread(image_path)        
        processed_image = apply_dwt_and_zero_coefficients_multidim(image, wavelet, percentage)
        processed_image = np.clip(processed_image, 0, 255).astype(np.uint8)
        output_filename = f'DWT_{wavelet}_{percentage}%_{image_filename}.png'
        cv2.imwrite(os.path.join(target_folder_path, output_filename), processed_image)


Apply FFT to the segmented image

In [ ]:
percentage = ...# choose from 1,5,20

dataset_path = os.path.join(HOME, 'Original_segmentation', 'train')
output_path = os.path.join(HOME, f'FFT_{percentage}%_segmentation', 'train')

os.makedirs(output_path, exist_ok=True)

for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)
    target_folder_path = os.path.join(output_path, folder_name)

    os.makedirs(target_folder_path, exist_ok=True)

    for image_filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_filename)
        image = cv2.imread(image_path)        
        filtered_image = fft_filter_on_color_image(image, percentage)
        output_filename = f'FFT_{percentage}%_{image_filename}.png'
        cv2.imwrite(os.path.join(target_folder_path, output_filename), filtered_image)


Create a huggingface dataset and push to hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
from datasets import load_dataset
data_dir = os.path.join(HOME, f'DWT_{wavelet}_{percentage}%_segmentation', 'train')
dataset = load_dataset("imagefolder", data_dir=data_dir)
wbc = dataset

In [ ]:
wbc.push_to_hub(f"Boom4869/DWT_{wavelet}_{percentage}per_WBC", private=True)